In [1]:
!pip install -q datasets

     |████████████████████████████████| 311 kB 5.2 MB/s 
     |████████████████████████████████| 133 kB 47.2 MB/s 
     |████████████████████████████████| 243 kB 16.3 MB/s 
     |████████████████████████████████| 1.1 MB 39.4 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 59.7 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 67.9 MB/s 


Code by clefourrier

In [2]:
from datasets import load_dataset
dataset_en = load_dataset('bigscience-historical-texts/hipe2020', 'en')
dataset_de = load_dataset('bigscience-historical-texts/hipe2020', 'de')
dataset_fr = load_dataset('bigscience-historical-texts/hipe2020', 'fr')

Downloading:   0%|          | 0.00/17.4k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/99.4k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset hipe2020 downloaded and prepared to /root/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/fr/1.0.0/e9899f43be4ecc44f52a23eae4e23d32ae9fee5710dac8ffeabe656e3c96ed0b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

# Data split

In [3]:
date_junction = range(1790, 1951, 20) 
period_dict_en = {i: [] for i in date_junction}
period_dict_de = {i: [] for i in date_junction}
period_dict_fr = {i: [] for i in date_junction}
date_counter = 0

for datasets, period_dict in [
    ([dataset_en["validation"]], period_dict_en),
    ([dataset_fr["validation"], dataset_fr["train"]], period_dict_fr),
    ([dataset_de["validation"], dataset_de["train"]], period_dict_de),
]:
    for dataset in datasets:
        dataset = dataset.sort("date")
        for item in dataset:
            # Our ranges are [start_year, end_year[
            if item['date'].year > 1949:
                date_counter = 0
                break
            while item['date'].year >= date_junction[date_counter + 1]:
                date_counter += 1

            period_dict[date_junction[date_counter]].append(item)
        date_counter = 0

# Latex table generation

In [4]:
for period in date_junction[:-1]:
    result = f"{period}-{period+20} & "
    for period_dict in [period_dict_en, period_dict_de, period_dict_fr]:
        period_list = period_dict[period]
        num_items = sum([len(i["NE_COARSE_LIT"]) for i in period_list])
        num_ner = sum(len(i["NE_COARSE_LIT"]) - i["NE_COARSE_LIT"].count(0) for i in period_list)
        if num_items == 0:
            result += f"{len(period_list)} & {num_items} & - & "
        else:
            result += f"{len(period_list)} & {num_items} & " + '{:2.1%}'.format(num_ner/num_items) + " & "
    print(result[:-2].replace("%", "\%") + "\\\\")

end_result = "Total & " 
for period_dict in [period_dict_en, period_dict_de, period_dict_fr]:
    num_items = sum([len(i["NE_COARSE_LIT"]) for period_list in period_dict.values() for i in period_list])
    num_ner = sum(len(i["NE_COARSE_LIT"]) - i["NE_COARSE_LIT"].count(0) for period_list in period_dict.values() for i in period_list)
    num_docs = sum([len(period_list) for period_list in period_dict.values()])
    end_result += f"{num_docs} & {num_items} & " + '{:2.1%}'.format(num_ner/num_items) + " & "
print(end_result[:-2].replace("%", "\%") + "\\\\")


1790-1810 & 10 & 4143 & 8.7\% & 13 & 6735 & 8.9\% & 14 & 8550 & 10.3\% \\
1810-1830 & 15 & 4697 & 8.9\% & 13 & 8049 & 6.4\% & 10 & 12440 & 14.1\% \\
1830-1850 & 9 & 3974 & 11.8\% & 19 & 15601 & 6.4\% & 10 & 11659 & 6.9\% \\
1850-1870 & 0 & 0 & - & 21 & 16021 & 7.3\% & 9 & 10321 & 8.9\% \\
1870-1890 & 7 & 2202 & 6.0\% & 16 & 17181 & 7.1\% & 15 & 16272 & 9.6\% \\
1890-1910 & 12 & 4509 & 8.6\% & 12 & 12829 & 10.8\% & 19 & 16874 & 10.3\% \\
1910-1930 & 13 & 5499 & 9.6\% & 13 & 18134 & 7.1\% & 30 & 30403 & 9.5\% \\
1930-1950 & 3 & 520 & 15.6\% & 29 & 24566 & 10.7\% & 32 & 35962 & 10.5\% \\
Total & 69 & 25544 & 9.3\% & 136 & 119116 & 8.2\% & 139 & 142481 & 10.1\% \\
